Purpose of this notebook is to setup a simple VAE that can learn poses of cubes.

@yvan june 9 2018

Setup

In [44]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils import data
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

In [30]:
# constants
NEPOCHS = 100
BATCH_SIZE = 64
SEED = 1337
IMG_DIM = 128
ncpus = !nproc
NCPUS = int(ncpus[0])
IMG_PATH = '/home/yvan/data_load/imgs_jpg_clpr/'

In [24]:
torch.cuda.is_available(), torch.__version__, torch.cuda.device_count()

(True, '0.4.0', 2)

In [25]:
cube_dataset = ImageFolder(IMG_PATH)
cube_loader = data.DataLoader(dataset=cube_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=NCPUS)

Create a simple VAE model.

In [43]:
(IMG_DIM*IMG_DIM)/50

327.68

In [54]:
torch.Tensor([-1,-2,-3,-4,-5,1,2,3,4,5])..exp_().log_()

tensor([ -2.,  -4.,  -6.,  -8., -10.,   2.,   4.,   6.,   8.,  10.])

In [48]:
torch.Tensor([1,2,3,4,5,6,7,8,9,10]).exp_()

tensor([     2.7183,      7.3891,     20.0855,     54.5981,    148.4132,
           403.4288,   1096.6332,   2980.9580,   8103.0840,  22026.4648])

In [55]:
# thanks to https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/Variational_autoencoder.py
# for his implementation of a simple VAE
class Vae(nn.Module):
    def __init__(self):
        super(Vae,self).__init__()
        # encoder dense layers
        self.dense1 = nn.Linear(IMG_DIM*IMG_DIM, int(IMG_DIM*IMG_DIM/50))
        # two output vectors, one for mu, and one for log variance.
        self.dense11 = nn.Linear(int(IMG_DIM*IMG_DIM/50), 25)
        self.dense12 = nn.Linear(int(IMG_DIM*IMG_DIM/50), 25)
        # decoder dense layers
        self.dense3 = nn.Linear(25, int(IMG_DIM*IMG_DIM/50))
        self.dense4 = nn.Linear(int(IMG_DIM*IMG_DIM/50), IMG_DIM*IMG_DIM)
        
    def encode(self, x):
        x = F.relu(self.dense1(x))
        return self.dense11(x), self.dense12(x)
    
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            epsilon = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            epsilon = torch.FloatTensor(std.size()).normal_()
        epsilon = Variable(epsilon)
        return epsilon.mul(std).add(mu)
    
    def decode(self, z):
        x = F.relu(sle.dense3(x))
        return F.sigmoid(self.dense4(x))
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [56]:
model = Vae()
if torch.cuda.is_available():
    model.cuda()
    
reconstruction_loss = nn.MSELoss(size_average=False)

def dual_loss_func(reconstructed, original, mu, logvar):
    mse = reconstruction_loss(reconstructed, original)
    kld = -0.5 * (1 + logvar - mu.pow(2) - logvar.exp())
    return mse + kld

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1524584710464/work/aten/src/THC/THCTensorRandom.cu:25